In [ ]:
import keras
print(keras.__version__)

In [ ]:
try:
  import google.colab
  IN_COLAB = True
  print('colab_env')
except:
  IN_COLAB = False
  print('local_env')

In [ ]:
if IN_COLAB:
  from google.colab import drive
  drive.mount('/content/gdrive')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
import json
import pickle
import tensorflow as tf
import os
from os import path
import argparse
import shutil
from itertools import repeat
from tensorflow.keras import layers
from keras.callbacks import ModelCheckpoint
from sklearn.utils import class_weight
from pathlib import Path
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, classification_report
import seaborn as sns
import random
from tensorflow import keras
from keras.models import load_model
import keras
import scipy.signal
import re

if IN_COLAB:
    !pip install wfdb
import wfdb

In [5]:
# Load the ECG signal and annotations from a JSON file
def load_ecg_data_from_json(file_path):
    with open(file_path, 'r') as infile:
        data = json.load(infile)
    signal = np.array(data['signal'])
    annotations = wfdb.Annotation(record_name=data['record_name'],
                                   extension=data['extension'],
                                   sample=data['sample'],
                                   symbol=data['symbol'],
                                   subtype=data['subtype'],
                                   chan=data['chan'],
                                   num=data['num'],
                                   aux_note=data['aux_note'])
    return signal, annotations

def load_ECG_MIT_BIH_mixedtestset_7_1_2_mod(half_window, overwrite, test, split, iswfdbdataset, dataset_path): # , session_path

  #dataset_path = "/media/paola/T7/Datasets/ECG_MITBIH/mit-bih-arrhythmia-database-1.0.0/"
  random_seed = 0# 60 #0 
  dataset_split = 0.7
  normalization = 0
  denoising = 1
  not_centered = 0
  base = 0

  augmentation = [0,1]#[16, 3]
  dataset_name = "NSVFQ"
  print("DATASET NAME IS ",dataset_name)
  data_names = ['100', '101', '103', '105', '106', #'107',
              '108', '109', '111', '112', '113', '114', '115', '116',
              '117', '118', '119', '121', '122', '123', '124', '200',
              '201', '202', '203', '205', '207', '208', '209', '210',
              '212', '213', '214', '215', '219', '220', '221', #'217', 
              '222', '223', '228', '230', '231', '232', '233', '234']

  test_set = []
  while 0:#(len(test_set)<5):
    test_record = random.choice(data_names)
    if test_record not in test_set:
      test_set.append(test_record)
  print("Test set is",test_set)
  test_record = 0
  a = 0
  b = 0
#############################################################################################################
#Create session
  temp = 's'
  #for s in subjects:
  #    temp += f'_{s}'
  #session_path = f"/content/gdrive/MyDrive/trained_models/model6_7_1_2_balanced/"
  #session_path = "output/train_raw/" + session_name + "/"

  # if test == 0:
  #   if os.path.isdir(session_path):
  #     if overwrite:
  #       try:
  #           shutil.rmtree(session_path)
  #           Path(session_path).mkdir(parents=True, exist_ok=True)
  #       except OSError:
  #           print("Error in session creation ("+session_path+").")
  #           exit()
  #     else:
  #       print(f'Session path ({session_path}) already exists')
  #       exit()
  #   else:
  #     try:
  #       Path(session_path).mkdir(parents=True, exist_ok=True)
  #     except OSError:
  #       print("Error in session creation ("+session_path+").")
  #       exit()
#############################################################################################################

  if dataset_name == 'NLRAV':
    labels = ['N', 'L', 'R', 'A', 'V']
    sub_labels = {'N':'N', 'L':'L', 'R':'R', 'A':'A', 'V':'V'}

  elif dataset_name == 'NSVFQ':
    labels = ['N', 'S', 'V', 'F', 'Q']
    sub_labels = { 'N':'N', 'L':'N', 'R':'N', 'e':'N', 'j':'N',
                   'A':'S', 'a':'S', 'J':'S', 'S':'S',
                   'V':'V', 'E':'V',
                   'F':'F',
                   '/':'Q', 'f':'Q', 'Q':'Q'}

  X = []
  Y = []
  C = []
  R = []
  RR = []
  # f_log = open(session_path + "dataload_log.txt", "w") 
  # f_log.write("Dataset " + dataset_name + "\n")


  if denoising == 1:
    denoise_fir = scipy.signal.firwin(13, 35, width=None, window='hamming', pass_zero='lowpass', scale=True, nyq=None, fs=360)
    print(denoise_fir)
    base = 6
  if not_centered == 1:
    with open('matrix_l_mod.pickle', 'rb') as input_file:
      matrix_l_mod = pk.load(input_file)

  for d in data_names:
    count_labels = np.zeros(5)

    # r = wfdb.rdrecord(dataset_path+d)
    # ann = wfdb.rdann(dataset_path+d, 'atr', return_label_elements=['label_store', 'symbol'])
    # if d!='114':
    #     sig = np.array(r.p_signal[:,0])
    #     intsig = np.array(r.p_signal[:,0])
    # else:
    #     sig = np.array(r.p_signal[:,1])
    #     intsig = np.array(r.p_signal[:,1])

    if iswfdbdataset:
      r = wfdb.rdrecord(dataset_path+d)
      ann = wfdb.rdann(dataset_path+d, 'atr', return_label_elements=['label_store', 'symbol'])
      if d!='114':
          sig = np.array(r.p_signal[:,0])
          intsig = np.array(r.p_signal[:,0])
      else:
          sig = np.array(r.p_signal[:,1])
          intsig = np.array(r.p_signal[:,1])
    else:
      sig, ann = load_ecg_data_from_json(dataset_path + d + '.json')
      if d != '114':
          sig = sig[:, 0]
      else:
          sig = sig[:, 1]

    if denoising == 1:
      basewond_sig = scipy.signal.medfilt(sig,71)
      basewond_sig2 = scipy.signal.medfilt(basewond_sig,215)
      sig = sig - basewond_sig2
      denoise_sig = scipy.signal.convolve(sig,denoise_fir)
    sig_len = len(sig)
    sym = ann.symbol
    pos = ann.sample
    if not_centered == 1:
      detected_pos = []
      label_pos = []
      file_index = data_names.index(d)
      matrix_line = matrix_l_mod[file_index]
      for element in matrix_line:
        if len(element) == 1:
          detected_pos.append(element[0][1])
          if detected_pos[-1] is not None:
            detected_pos[-1] = detected_pos[-1] + base
          label_pos.append(element[0][0] + base)
        else:
          print("Double peak detected, choosing first")
          detected_pos.append(element[0][1])
          if detected_pos[-1] is not None:
            detected_pos[-1] = detected_pos[-1] + base
          label_pos.append(element[0][0] + base)
          print(detected_pos[-1])
      detected_pos = np.array(detected_pos)
    if denoising == 1:
     for i in range(len(pos)):
       pos[i] = pos[i] + base #adjust delay connected to powerline denoising with 12-order FIR filter
     sig = denoise_sig
    beat_len = len(sym)
    pre = None
    detected_index = 0
    #base =  0
    if d not in test_set:
      print("Appending data in " + d + " to train data")
      # f_log.write("Appending data in " + d + " to train data\n")
      for i in range(beat_len):
        for j in range(-augmentation[0]*augmentation[1],augmentation[0]*augmentation[1]+1,augmentation[1]):
            if pos[i]-half_window+j>=base and pos[i]+half_window+j<=(sig_len+base) and sym[i] in sub_labels:
                if not_centered == 1:
                    if detected_pos[detected_index] is not None:
                      if detected_pos[detected_index]+half_window+j > (sig_len+base):
                        frame = sig[detected_pos[detected_index]-half_window+j:sig_len+base]
                      elif detected_pos[detected_index]-half_window+j < base:
                        frame = sig[base:detected_pos[detected_index]+half_window+j]
                      else:
                        frame = sig[detected_pos[detected_index]-half_window+j:detected_pos[detected_index]+half_window+j]
                      print(d, detected_pos[detected_index], pos[i])
#                      print(detected_pos[detected_index]-half_window,detected_pos[detected_index]+half_window+j, len(frame))
                      detected_index = detected_index + 1
                    else:
                      frame = np.zeros(198)
                      detected_index = detected_index + 1
                    X.append(frame)
               #     print(a)
                    a = a + 1
                    if detected_pos[detected_index-1] is None or len(frame)<198:
                      C.append(False)   # if the peak is not detected remove from testset after split
                      X[-1] = sig[pos[i]-half_window+j:pos[i]+half_window+j]
                    elif pre is not None:
                      if pos[i]-pre<half_window:
                        print("ATTENTION, OVERLAPPING WINDOWS")
                        print(pre,pos[i])
                      C.append(True if (j == 0 and (pos[i]-pre)>half_window)  else False)
                    else:
                      C.append(True if (j == 0) else False)
                    R.append([data_names.index(d),pos[i]-base, detected_pos[detected_index-1]])
                    post = 0
                    index = i
                    # eval postRR
                    if detected_index < len(detected_pos):
                      post = detected_pos[detected_index] if detected_pos[detected_index] is not None else None
                    else:
                      post = None 
                    if post is None or detected_pos[detected_index-1] is None: # if next peek or current peak is not detected set to maximum distance
                      postRR = 500
                    elif post-detected_pos[detected_index-1] > 500:
                      postRR = 500
                    else:
                      postRR = post-detected_pos[detected_index-1]
                    # eval preRR
                    if detected_index >=2:
                      pre = detected_pos[detected_index-2]
                    else:
                      pre = base # for the first pek consider the start of the record
                    if pre is None or detected_pos[detected_index - 1] is None: # if previous peek or current peak is not detected set to maximum distance
                      preRR = 500
                    elif detected_pos[detected_index-1]-pre > 500:
                      preRR = 500
                    else:
                     preRR = detected_pos[detected_index-1]-pre
                elif not_centered == 0:
                  frame = sig[pos[i]-half_window+j:pos[i]+half_window+j]
                  X.append(frame)
                #  print(a)
                  a = a + 1
                  if pre is not None:
                   if pos[i]-pre<half_window:
                     print("ATTENTION, OVERLAPPING WINDOWS")
                     print(pre,pos[i])
                   C.append(True if (j == 0 and (pos[i]-pre)>half_window)  else False)
                  else:
                   C.append(True if (j == 0) else False)
                  R.append([data_names.index(d),pos[i]])
                  post = 0
                  index = i
                  while post == 0 and index<(beat_len-1):
                    if sym[index+1] in sub_labels:
                      post = pos[index+1]
                    else:
                      index = index + 1
                  if pre is None:
                      preRR = pos[i]
                  elif pos[i]-pre > 500:
                      preRR = 500
                  else:
                     preRR = pos[i]-pre
                  if post-pos[i] > 500:
                     postRR = 500
                  else:
                     postRR = post-pos[i]
                Y.append(labels.index(sub_labels[sym[i]]))
                count_labels[labels.index(sub_labels[sym[i]])] = count_labels[labels.index(sub_labels[sym[i]])] + 1
                RR.append([preRR,postRR])
                pre = pos[i]
            elif not_centered == 1 and pos[i] == label_pos[detected_index] :  # skip the first detected peak if needed to match the first admissible window based on label peaks
              detected_index = detected_index +  1
    print("Patient " + d + " includes labels " + str(int(count_labels[0])) + " " + str(int(count_labels[1])) + " " + str(int(count_labels[2])) + " " + str(int(count_labels[3])) + " " + str(int(count_labels[4])) )
    # f_log.write("Patient " + d + " includes labels " + str(int(count_labels[0])) + " " + str(int(count_labels[1])) + " " + str(int(count_labels[2])) + " " + str(int(count_labels[3])) + " " + str(int(count_labels[4])))
    # f_log.write("\n")
  item_perm = np.arange(np.size(X,0))
  np.random.seed(random_seed)
  np.random.shuffle(item_perm)

#  f = open(session_path + "R_all.txt","w")
 # for i in np.array(R):
 #   s = str(i[0]) + " " + str(i[1]) + "\n"#" " + str(i[2]) + "\n"
 #   f.write(s)
 # f.close()
  print(np.array(X).shape)

  X = np.array(X)[item_perm]
  Y = np.array(Y)[item_perm]
  C = np.array(C)[item_perm]
  R = np.array(R)[item_perm]
  RR = np.array(RR)[item_perm]

  print(split)
  if split == 1:
    X_train = X[:round(np.size(X,0)*dataset_split)]
    Y_train = Y[:round(np.size(X,0)*dataset_split)]
    RR_train = RR[:round(np.size(X,0)*dataset_split)]
  # C_train = C[:round(np.size(X,0)*dataset_split)]
    R_train = R[:round(np.size(X,0)*dataset_split)]
  # P_train = P[:round(np.size(X,0)*dataset_split)]

    X_valid_ = X[round(np.size(X,0)*dataset_split):round(np.size(X,0)*0.8)] 
    Y_valid_ = Y[round(np.size(X,0)*dataset_split):round(np.size(X,0)*0.8)]
    C_valid = C[round(np.size(X,0)*dataset_split):round(np.size(X,0)*0.8)]
    R_valid = R[round(np.size(X,0)*dataset_split):round(np.size(X,0)*0.8)]
    RR_valid = RR[round(np.size(X,0)*dataset_split):round(np.size(X,0)*0.8)]
                                                        
    X_test_ = X[round(np.size(X,0)*0.8):] 
    Y_test_ = Y[round(np.size(X,0)*0.8):]
    C_test = C[round(np.size(X,0)*0.8):]
    R_test = R[round(np.size(X,0)*0.8):]
    RR_test = RR[round(np.size(X,0)*0.8):]

  elif split == 2:
    X_test_ = X[:round(np.size(X,0)*0.2)] 
    Y_test_ = Y[:round(np.size(X,0)*0.2)]
    C_test = C[:round(np.size(X,0)*0.2)]
    R_test = R[:round(np.size(X,0)*0.2)]
    RR_test = RR[:round(np.size(X,0)*0.2)]

    X_valid_ = X[round(np.size(X,0)*0.2):round(np.size(X,0)*0.3)] 
    Y_valid_ = Y[round(np.size(X,0)*0.2):round(np.size(X,0)*0.3)]
    C_valid = C[round(np.size(X,0)*0.2):round(np.size(X,0)*0.3)]
    R_valid = R[round(np.size(X,0)*0.2):round(np.size(X,0)*0.3)]
    RR_valid = RR[round(np.size(X,0)*0.2):round(np.size(X,0)*0.3)]

    X_train = X[round(np.size(X,0)*0.3):]
    Y_train = Y[round(np.size(X,0)*0.3):]
    RR_train = RR[round(np.size(X,0)*0.3):]
  # C_train = C[:round(np.size(X,0)*dataset_split)]
    R_train = R[round(np.size(X,0)*0.3):]
  # P_train = P[:round(np.size(X,0)*dataset_split)]

  elif split == 3:
    X_test_ = X[round(np.size(X,0)*0.2):round(np.size(X,0)*0.4)] 
    Y_test_ = Y[round(np.size(X,0)*0.2):round(np.size(X,0)*0.4)]
    C_test = C[round(np.size(X,0)*0.2):round(np.size(X,0)*0.4)]
    R_test = R[round(np.size(X,0)*0.2):round(np.size(X,0)*0.4)]
    RR_test = RR[round(np.size(X,0)*0.2):round(np.size(X,0)*0.4)]

    X_valid_ = X[round(np.size(X,0)*0.4):round(np.size(X,0)*0.5)] 
    Y_valid_ = Y[round(np.size(X,0)*0.4):round(np.size(X,0)*0.5)]
    C_valid = C[round(np.size(X,0)*0.4):round(np.size(X,0)*0.5)]
    R_valid = R[round(np.size(X,0)*0.4):round(np.size(X,0)*0.5)]
    RR_valid = RR[round(np.size(X,0)*0.4):round(np.size(X,0)*0.5)]

    X_train = X[:round(np.size(X,0)*0.2)]
    Y_train = Y[:round(np.size(X,0)*0.2)]
    RR_train = RR[:round(np.size(X,0)*0.2)]
    R_train = R[:round(np.size(X,0)*0.2)]

    X_train = np.append(X_train, X[round(np.size(X,0)*0.5):], axis = 0)
    Y_train = np.append(Y_train, Y[round(np.size(X,0)*0.5):], axis = 0)
    RR_train = np.append(RR_train, RR[round(np.size(X,0)*0.5):], axis = 0)
    R_train = np.append(R_train, R[round(np.size(X,0)*0.5):], axis = 0)

  elif split == 4:
    X_test_ = X[round(np.size(X,0)*0.4):round(np.size(X,0)*0.6)] 
    Y_test_ = Y[round(np.size(X,0)*0.4):round(np.size(X,0)*0.6)]
    C_test = C[round(np.size(X,0)*0.4):round(np.size(X,0)*0.6)]
    R_test = R[round(np.size(X,0)*0.4):round(np.size(X,0)*0.6)]
    RR_test = RR[round(np.size(X,0)*0.4):round(np.size(X,0)*0.6)]

    X_valid_ = X[round(np.size(X,0)*0.6):round(np.size(X,0)*0.7)] 
    Y_valid_ = Y[round(np.size(X,0)*0.6):round(np.size(X,0)*0.7)]
    C_valid = C[round(np.size(X,0)*0.6):round(np.size(X,0)*0.7)]
    R_valid = R[round(np.size(X,0)*0.6):round(np.size(X,0)*0.7)]
    RR_valid = RR[round(np.size(X,0)*0.6):round(np.size(X,0)*0.7)]

    X_train = X[:round(np.size(X,0)*0.4)] 
    Y_train = Y[:round(np.size(X,0)*0.4)]
    RR_train = RR[:round(np.size(X,0)*0.4)]
    R_train = R[:round(np.size(X,0)*0.4)]

    X_train = np.append(X_train, X[round(np.size(X,0)*0.7):], axis = 0)
    Y_train = np.append(Y_train, Y[round(np.size(X,0)*0.7):], axis = 0)
    RR_train = np.append(RR_train, RR[round(np.size(X,0)*0.7):], axis = 0)
    R_train = np.append(R_train, R[round(np.size(X,0)*0.7):], axis = 0)

  elif split == 5:
    X_test_ = X[round(np.size(X,0)*0.6):round(np.size(X,0)*0.8)] 
    Y_test_ = Y[round(np.size(X,0)*0.6):round(np.size(X,0)*0.8)]
    C_test = C[round(np.size(X,0)*0.6):round(np.size(X,0)*0.8)]
    R_test = R[round(np.size(X,0)*0.6):round(np.size(X,0)*0.8)]
    RR_test = RR[round(np.size(X,0)*0.6):round(np.size(X,0)*0.8)]

    X_valid_ = X[round(np.size(X,0)*0.8):round(np.size(X,0)*0.9)] 
    Y_valid_ = Y[round(np.size(X,0)*0.8):round(np.size(X,0)*0.9)]
    C_valid = C[round(np.size(X,0)*0.8):round(np.size(X,0)*0.9)]
    R_valid = R[round(np.size(X,0)*0.8):round(np.size(X,0)*0.9)]
    RR_valid = RR[round(np.size(X,0)*0.8):round(np.size(X,0)*0.9)]

    X_train = X[:round(np.size(X,0)*0.6)]
    Y_train = Y[:round(np.size(X,0)*0.6)]
    RR_train = RR[:round(np.size(X,0)*0.6)]
    R_train = R[:round(np.size(X,0)*0.6)]

    X_train = np.append(X_train, X[round(np.size(X,0)*0.9):], axis = 0)
    Y_train = np.append(Y_train, Y[round(np.size(X,0)*0.9):], axis = 0)
    RR_train = np.append(RR_train, RR[round(np.size(X,0)*0.9):], axis = 0)
    R_train = np.append(R_train, R[round(np.size(X,0)*0.9):], axis = 0)

  X_valid = X_valid_[C_valid]
  Y_valid = Y_valid_[C_valid]
  RR_valid = RR_valid[C_valid]
  X_test = X_test_[C_test]
  Y_test = Y_test_[C_test]
  RR_test = RR_test[C_test]
  R_valid = R_valid[C_valid]
  R_test = R_test[C_test]
 # f = open(session_path + "R_detected.txt","w")
 # for i in R_test:
 #   s = str(i[0]) + " " + str(i[1]) + " " + str(i[2]) +"\n"
 #   f.write(s)
 # f.close()

  print("Validation set",X_valid.shape)
  print("Test set",X_test.shape)
  if normalization:
    for i in range(np.size(X_train,0)):
        X_train[i]=X_train[i]/np.max(np.absolute(X_train[i]))
    for i in range(np.size(X_valid,0)):
        X_valid[i]=X_valid[i]/np.max(np.absolute(X_valid[i]))
    for i in range(np.size(X_test,0)):
        X_test[i]=X_test[i]/np.max(np.absolute(X_test[i]))

  X_train = X_train.reshape(X_train.shape[0],X_train.shape[1],1,1)
  X_valid = X_valid.reshape(X_valid.shape[0],X_valid.shape[1],1,1)
  X_test = X_test.reshape(X_test.shape[0],X_test.shape[1],1,1)

  print("X train_shape",X_train.shape)
  print("X valid_shape",X_valid.shape)
  print("Y train shape", Y_train.shape)
  print("Y valid shape", Y_valid.shape)
  print("X_test shape", X_test.shape)
  print("Y test shape", Y_test.shape)

  print("RR_train", RR_train.shape)
  print("RR_valid", RR_valid.shape)
  print("RR_test", RR_test.shape)

  class_types = labels
  # f_log.close()

  return X_train, Y_train, X_valid, Y_valid, X_test, Y_test, R_train, R_valid, R_test, RR_train, RR_valid, RR_test, class_types


def load_ECG_MIT_BIH_mixedtestset_7_1_2(half_window, session_name, overwrite, test, split, in_colab):

  wfdbdataset = False

  if wfdbdataset:
    if in_colab:
      dataset_path = "/content/gdrive/MyDrive/dataset/mit-bih-arrhythmia-database-1.0.0/"
    else:
      dataset_path = "../../dataset/mit-bih-arrhythmia-database-1.0.0/"
  else:
    if in_colab:
      dataset_path = "/content/gdrive/MyDrive/dataset/mit-bih-arrhythmia-database-1.0.0_em24/"
    else:
      dataset_path = "../../dataset/mit-bih-arrhythmia-database-1.0.0_em24/"

  random_seed = 0# 60 #0 
  dataset_split = 0.7
  normalization = 0
  denoising = 1
  base = 0

  augmentation = [0,1]#[16, 3]
  dataset_name = "NSVFQ"
  print("DATASET NAME IS ",dataset_name)
  data_names = ['100', '101', '103', '105', '106', #'107',
              '108', '109', '111', '112', '113', '114', '115', '116',
              '117', '118', '119', '121', '122', '123', '124', '200',
              '201', '202', '203', '205', '207', '208', '209', '210',
              '212', '213', '214', '215', '219', '220', '221', #'217', 
              '222', '223', '228', '230', '231', '232', '233', '234']

  test_set = []
  while 0:#(len(test_set)<5):
    test_record = random.choice(data_names)
    if test_record not in test_set:
      test_set.append(test_record)
  print("Test set is",test_set)
  test_record = 0


#############################################################################################################
#Create session
  if in_colab:
    session_path = f"/content/gdrive/MyDrive/output/train/em24/"
  else:
    session_path = f"output/train/test_noise/"

  os.makedirs(session_path, exist_ok=True)
 
#############################################################################################################


  if dataset_name == 'NLRAV':
    labels = ['N', 'L', 'R', 'A', 'V']
    sub_labels = {'N':'N', 'L':'L', 'R':'R', 'A':'A', 'V':'V'}

  elif dataset_name == 'NSVFQ':
    labels = ['N', 'S', 'V', 'F', 'Q']
    sub_labels = { 'N':'N', 'L':'N', 'R':'N', 'e':'N', 'j':'N',
                   'A':'S', 'a':'S', 'J':'S', 'S':'S',
                   'V':'V', 'E':'V',
                   'F':'F',
                   '/':'Q', 'f':'Q', 'Q':'Q'}

  X = []
  Y = []
  C = []
  R = []
  RR = []
  f_log = open(session_path + "dataload_log.txt", "w") 
  f_log.write("Dataset " + dataset_name + "\n")

  if denoising == 1:
    denoise_fir = scipy.signal.firwin(13, 35, width=None, window='hamming', pass_zero='lowpass', scale=True, nyq=None, fs=360)
    print(denoise_fir)
    base = 6

  for d in data_names:
    count_labels = np.zeros(5)
    if wfdbdataset:
      r = wfdb.rdrecord(dataset_path+d)
      ann = wfdb.rdann(dataset_path+d, 'atr', return_label_elements=['label_store', 'symbol'])
      if d!='114':
          sig = np.array(r.p_signal[:,0])
          intsig = np.array(r.p_signal[:,0])
      else:
          sig = np.array(r.p_signal[:,1])
          intsig = np.array(r.p_signal[:,1])
    else:
      sig, ann = load_ecg_data_from_json(dataset_path + d + '.json')
      if d != '114':
          sig = sig[:, 0]
      else:
          sig = sig[:, 1]
    if denoising == 1:
      print(sig.shape)
      plt.plot(sig[:4000],label="original")
      basewond_sig = scipy.signal.medfilt(sig,71)
      basewond_sig2 = scipy.signal.medfilt(basewond_sig,215)
      #sig = sig - basewond_sig
      sig = sig - basewond_sig2
      denoise_sig = scipy.signal.convolve(sig,denoise_fir)
      plt.plot(denoise_sig[6:4006], label="denoised")
      plt.legend()
      plt.show()
    sig_len = len(sig)
    sym = ann.symbol
    pos = ann.sample
    beat_len = len(sym)
    if denoising == 1:
      for i in range(len(pos)):
         pos[i] = pos[i] + base #adjust delay connected to powerline denoising with 12-order FIR filter
      sig = denoise_sig
    pre = None
    if d not in test_set:
      print("Appending data in " + d + " to train data")
      f_log.write("Appending data in " + d + " to train data\n")
      for i in range(beat_len):
        for j in range(-augmentation[0]*augmentation[1],augmentation[0]*augmentation[1]+1,augmentation[1]):
            if pos[i]-half_window+j>=base and pos[i]+half_window+j<=(sig_len+base) and sym[i] in sub_labels:
                frame = sig[pos[i]-half_window+j:pos[i]+half_window+j]
                X.append(frame)
                Y.append(labels.index(sub_labels[sym[i]]))
                count_labels[labels.index(sub_labels[sym[i]])] = count_labels[labels.index(sub_labels[sym[i]])] + 1
                if pre is not None:
                   if pos[i]-pre<half_window:
                     print("ATTENTION, OVERLAPPING WINDOWS")
                     print(pre,pos[i])
                   C.append(True if (j == 0 and (pos[i]-pre)>half_window) else False)
                else:
                   C.append(True if (j == 0) else False)
                R.append(data_names.index(d))
                post = 0
                index = i
                while post == 0 and index<(beat_len-1):
                  if sym[index+1] in sub_labels:
                    post = pos[index+1]
                  else:
                    index = index + 1
                if pre is None:
                   preRR = pos[i]
                elif pos[i]-pre > 500:
                   preRR = 500
                  # print("More than 500 Pre")
                else:
                   preRR = pos[i]-pre
                if post-pos[i] > 500:
                   postRR = 500
                 #  print("More than 500 Post")
                else:
                   postRR = post-pos[i]
                RR.append([preRR,postRR])
                pre = pos[i]
    print("Patient " + d + " includes labels " + str(int(count_labels[0])) + " " + str(int(count_labels[1])) + " " + str(int(count_labels[2])) + " " + str(int(count_labels[3])) + " " + str(int(count_labels[4])) )
    f_log.write("Patient " + d + " includes labels " + str(int(count_labels[0])) + " " + str(int(count_labels[1])) + " " + str(int(count_labels[2])) + " " + str(int(count_labels[3])) + " " + str(int(count_labels[4])))
    f_log.write("\n")
  item_perm = np.arange(np.size(X,0))

  np.random.seed(random_seed)
  np.random.shuffle(item_perm)

  X = np.array(X)[item_perm]
  Y = np.array(Y)[item_perm]
  C = np.array(C)[item_perm]
  R = np.array(R)[item_perm]
  RR = np.array(RR)[item_perm]

  if split == 1:
    print("Split 1")
    X_train = X[:round(np.size(X,0)*dataset_split)]
    Y_train = Y[:round(np.size(X,0)*dataset_split)]
    RR_train = RR[:round(np.size(X,0)*dataset_split)]
  # C_train = C[:round(np.size(X,0)*dataset_split)]
    R_train = R[:round(np.size(X,0)*dataset_split)]
  # P_train = P[:round(np.size(X,0)*dataset_split)]

    X_valid_ = X[round(np.size(X,0)*dataset_split):round(np.size(X,0)*0.8)] 
    Y_valid_ = Y[round(np.size(X,0)*dataset_split):round(np.size(X,0)*0.8)]
    C_valid = C[round(np.size(X,0)*dataset_split):round(np.size(X,0)*0.8)]
    R_valid = R[round(np.size(X,0)*dataset_split):round(np.size(X,0)*0.8)]
    RR_valid = RR[round(np.size(X,0)*dataset_split):round(np.size(X,0)*0.8)]
                                                        
    X_test_ = X[round(np.size(X,0)*0.8):] 
    Y_test_ = Y[round(np.size(X,0)*0.8):]
    C_test = C[round(np.size(X,0)*0.8):]
    R_test = R[round(np.size(X,0)*0.8):]
    RR_test = RR[round(np.size(X,0)*0.8):]

  elif split == 2:
    print("Split 2")
    X_test_ = X[:round(np.size(X,0)*0.2)] 
    Y_test_ = Y[:round(np.size(X,0)*0.2)]
    C_test = C[:round(np.size(X,0)*0.2)]
    R_test = R[:round(np.size(X,0)*0.2)]
    RR_test = RR[:round(np.size(X,0)*0.2)]

    X_valid_ = X[round(np.size(X,0)*0.2):round(np.size(X,0)*0.3)] 
    Y_valid_ = Y[round(np.size(X,0)*0.2):round(np.size(X,0)*0.3)]
    C_valid = C[round(np.size(X,0)*0.2):round(np.size(X,0)*0.3)]
    R_valid = R[round(np.size(X,0)*0.2):round(np.size(X,0)*0.3)]
    RR_valid = RR[round(np.size(X,0)*0.2):round(np.size(X,0)*0.3)]

    X_train = X[round(np.size(X,0)*0.3):]
    Y_train = Y[round(np.size(X,0)*0.3):]
    RR_train = RR[round(np.size(X,0)*0.3):]
  # C_train = C[:round(np.size(X,0)*dataset_split)]
    R_train = R[round(np.size(X,0)*0.3):]
  # P_train = P[:round(np.size(X,0)*dataset_split)]

  elif split == 3:
    print("Split 3")
    X_test_ = X[round(np.size(X,0)*0.2):round(np.size(X,0)*0.4)] 
    Y_test_ = Y[round(np.size(X,0)*0.2):round(np.size(X,0)*0.4)]
    C_test = C[round(np.size(X,0)*0.2):round(np.size(X,0)*0.4)]
    R_test = R[round(np.size(X,0)*0.2):round(np.size(X,0)*0.4)]
    RR_test = RR[round(np.size(X,0)*0.2):round(np.size(X,0)*0.4)]

    X_valid_ = X[round(np.size(X,0)*0.4):round(np.size(X,0)*0.5)] 
    Y_valid_ = Y[round(np.size(X,0)*0.4):round(np.size(X,0)*0.5)]
    C_valid = C[round(np.size(X,0)*0.4):round(np.size(X,0)*0.5)]
    R_valid = R[round(np.size(X,0)*0.4):round(np.size(X,0)*0.5)]
    RR_valid = RR[round(np.size(X,0)*0.4):round(np.size(X,0)*0.5)]

    X_train = X[:round(np.size(X,0)*0.2)]
    Y_train = Y[:round(np.size(X,0)*0.2)]
    RR_train = RR[:round(np.size(X,0)*0.2)]
    R_train = R[:round(np.size(X,0)*0.2)]

    X_train = np.append(X_train, X[round(np.size(X,0)*0.5):], axis = 0)
    Y_train = np.append(Y_train, Y[round(np.size(X,0)*0.5):], axis = 0)
    RR_train = np.append(RR_train, RR[round(np.size(X,0)*0.5):], axis = 0)
    R_train = np.append(R_train, R[round(np.size(X,0)*0.5):], axis = 0)

  elif split == 4:
    print("Split 4")
    X_test_ = X[round(np.size(X,0)*0.4):round(np.size(X,0)*0.6)] 
    Y_test_ = Y[round(np.size(X,0)*0.4):round(np.size(X,0)*0.6)]
    C_test = C[round(np.size(X,0)*0.4):round(np.size(X,0)*0.6)]
    R_test = R[round(np.size(X,0)*0.4):round(np.size(X,0)*0.6)]
    RR_test = RR[round(np.size(X,0)*0.4):round(np.size(X,0)*0.6)]

    X_valid_ = X[round(np.size(X,0)*0.6):round(np.size(X,0)*0.7)] 
    Y_valid_ = Y[round(np.size(X,0)*0.6):round(np.size(X,0)*0.7)]
    C_valid = C[round(np.size(X,0)*0.6):round(np.size(X,0)*0.7)]
    R_valid = R[round(np.size(X,0)*0.6):round(np.size(X,0)*0.7)]
    RR_valid = RR[round(np.size(X,0)*0.6):round(np.size(X,0)*0.7)]

    X_train = X[:round(np.size(X,0)*0.4)] 
    Y_train = Y[:round(np.size(X,0)*0.4)]
    RR_train = RR[:round(np.size(X,0)*0.4)]
    R_train = R[:round(np.size(X,0)*0.4)]

    X_train = np.append(X_train, X[round(np.size(X,0)*0.7):], axis = 0)
    Y_train = np.append(Y_train, Y[round(np.size(X,0)*0.7):], axis = 0)
    RR_train = np.append(RR_train, RR[round(np.size(X,0)*0.7):], axis = 0)
    R_train = np.append(R_train, R[round(np.size(X,0)*0.7):], axis = 0)

  elif split == 5:
    print("Split 5")
    X_test_ = X[round(np.size(X,0)*0.6):round(np.size(X,0)*0.8)] 
    Y_test_ = Y[round(np.size(X,0)*0.6):round(np.size(X,0)*0.8)]
    C_test = C[round(np.size(X,0)*0.6):round(np.size(X,0)*0.8)]
    R_test = R[round(np.size(X,0)*0.6):round(np.size(X,0)*0.8)]
    RR_test = RR[round(np.size(X,0)*0.6):round(np.size(X,0)*0.8)]

    X_valid_ = X[round(np.size(X,0)*0.8):round(np.size(X,0)*0.9)] 
    Y_valid_ = Y[round(np.size(X,0)*0.8):round(np.size(X,0)*0.9)]
    C_valid = C[round(np.size(X,0)*0.8):round(np.size(X,0)*0.9)]
    R_valid = R[round(np.size(X,0)*0.8):round(np.size(X,0)*0.9)]
    RR_valid = RR[round(np.size(X,0)*0.8):round(np.size(X,0)*0.9)]

    X_train = X[:round(np.size(X,0)*0.6)]
    Y_train = Y[:round(np.size(X,0)*0.6)]
    RR_train = RR[:round(np.size(X,0)*0.6)]
    R_train = R[:round(np.size(X,0)*0.6)]

    X_train = np.append(X_train, X[round(np.size(X,0)*0.9):], axis = 0)
    Y_train = np.append(Y_train, Y[round(np.size(X,0)*0.9):], axis = 0)
    RR_train = np.append(RR_train, RR[round(np.size(X,0)*0.9):], axis = 0)
    R_train = np.append(R_train, R[round(np.size(X,0)*0.9):], axis = 0)

  X_valid = X_valid_[C_valid]
  Y_valid = Y_valid_[C_valid]
  RR_valid = RR_valid[C_valid]
  X_test = X_test_[C_test]
  Y_test = Y_test_[C_test]
  RR_test = RR_test[C_test]
  R_valid = R_valid[C_valid]
  R_test = R_test[C_test]

  print("Validation set",X_valid.shape)
  print("Test set",X_test.shape)
  if normalization:
  ############# old normalization matteo #################
  #  for i in range(np.size(X_train,0)):
  #      X_train[i]=X_train[i]/np.max(np.absolute(X_train[i]))
  #  for i in range(np.size(X_valid,0)):
  #      X_valid[i]=X_valid[i]/np.max(np.absolute(X_valid[i]))
  #  for i in range(np.size(X_test,0)):
  #      X_test[i]=X_test[i]/np.max(np.absolute(X_test[i])) 

  ############# normalization paola #################
   # max_training = np.max(np.absolute(X_train))
    #for i in range(np.size(X_train,0)):
   #     X_train[i]=X_train[i]/max_training
   # for i in range(np.size(X_valid,0)):
     #   X_valid[i]=X_valid[i]/max_training
   # for i in range(np.size(X_test,0)):
     #   X_test[i]=X_test[i]/max_training

############# normalization RR #################
    for i in range(np.size(RR_train,0)):
        RR_train[i][0]=RR_train[i][0]/np.mean(RR_train)
        RR_train[i][1]=RR_train[i][1]/np.mean(RR_train)
    for i in range(np.size(RR_valid,0)):
        RR_valid[i][0]=RR_valid[i][0]/np.mean(RR_valid)
        RR_valid[i][1]=RR_valid[i][1]/np.mean(RR_valid)
    for i in range(np.size(RR_test,0)):
        RR_test[i][0]=RR_test[i][0]/np.mean(RR_test)
        RR_test[i][1]=RR_test[i][1]/np.mean(RR_test)

  X_train = X_train.reshape(X_train.shape[0],X_train.shape[1],1,1)
  X_valid = X_valid.reshape(X_valid.shape[0],X_valid.shape[1],1,1)
  X_test = X_test.reshape(X_test.shape[0],X_test.shape[1],1,1)

  print("X train_shape",X_train.shape)
  print("X valid_shape",X_valid.shape)
  print("Y train shape", Y_train.shape)
  print("Y valid shape", Y_valid.shape)
  print("X_test shape", X_test.shape)
  print("Y test shape", Y_test.shape)

  class_types = labels
  f_log.close()

  return X_train, Y_train, X_valid, Y_valid, X_test, Y_test, R_train, R_valid, R_test, RR_train, RR_valid, RR_test, class_types, session_path

In [6]:
session_name = 'train_session'
kernel = 3
emb_depth = 1
embedding = 16
transformer_layers = 1
hidden_size = 2 # key dim in attention layer
num_heads =  8
mlp_dim = 128
overwrite = 1
random_seed=0
batch_size=128
half_window = 99
use_cls_token = 0
split = 1
cat_RR = 1

In [ ]:
#############################################################################################################
#Create session

if IN_COLAB:
    session_path = f"/content/gdrive/MyDrive/output/train/em_3_10_24/"
else:
    session_path = f"output/train/em_3_10_24/"

os.makedirs(session_path, exist_ok=True)
 
#############################################################################################################

#############################################################################################################
#Load dataset ECG

# x_train, y_train, x_valid, y_valid, x_test, y_test, R_train, R_valid, R_test, RR_train, RR_valid, RR_test, class_types, session_path = load_ECG_MIT_BIH_mixedtestset_7_1_2(half_window, session_name, overwrite,1,split,IN_COLAB)

if IN_COLAB:
    dataset_path = "/content/gdrive/MyDrive/dataset/mit-bih-arrhythmia-database-1.0.0/"
    other_dataset_path = ["/content/gdrive/MyDrive/dataset/mit-bih-arrhythmia-database-1.0.0_em3/",
                          "/content/gdrive/MyDrive/dataset/mit-bih-arrhythmia-database-1.0.0_em10/",
                          "/content/gdrive/MyDrive/dataset/mit-bih-arrhythmia-database-1.0.0_em24/",
                          ]
else:
    dataset_path = "../../dataset/mit-bih-arrhythmia-database-1.0.0/"
    other_dataset_path = ["../../dataset/mit-bih-arrhythmia-database-1.0.0_em3/",
                          "../../dataset/mit-bih-arrhythmia-database-1.0.0_em10/",
                          "../../dataset/mit-bih-arrhythmia-database-1.0.0_em24/",
                          ]

if dataset_path.endswith("mit-bih-arrhythmia-database-1.0.0/"):
    iswfdbdataset = True
else:
    iswfdbdataset = False

x_train, y_train, x_valid, y_valid, x_test, y_test,  R_train, R_valid,R_test, RR_train, RR_valid, RR_test, class_types = load_ECG_MIT_BIH_mixedtestset_7_1_2_mod(half_window, overwrite, 1, split, iswfdbdataset, dataset_path) # , session_path
 
for o_dataset_path in other_dataset_path:
    if o_dataset_path.endswith("mit-bih-arrhythmia-database-1.0.0/"):
        iswfdbdataset = True
    else:
        iswfdbdataset = False
    o_x_train, o_y_train, o_x_valid, o_y_valid, o_x_test, o_y_test,  o_R_train, o_R_valid, o_R_test, o_RR_train, o_RR_valid, o_RR_test, o_class_types = load_ECG_MIT_BIH_mixedtestset_7_1_2_mod(half_window, overwrite, 1, split, iswfdbdataset, o_dataset_path) # , session_path
    x_train = np.concatenate((x_train, o_x_train), axis=0)
    y_train = np.concatenate((y_train, o_y_train), axis=0)
    x_valid = np.concatenate((x_valid, o_x_valid), axis=0)
    y_valid = np.concatenate((y_valid, o_y_valid), axis=0)
    x_test = np.concatenate((x_test, o_x_test), axis=0)
    y_test = np.concatenate((y_test, o_y_test), axis=0)
    R_train = np.concatenate((R_train, o_R_train), axis=0)
    R_valid = np.concatenate((R_valid, o_R_valid), axis=0)
    R_test = np.concatenate((R_test, o_R_test), axis=0)
    RR_train = np.concatenate((RR_train, o_RR_train), axis=0)
    RR_valid = np.concatenate((RR_valid, o_RR_valid), axis=0)
    RR_test = np.concatenate((RR_test, o_RR_test), axis=0)

print("Final x_train shape:", x_train.shape)
print("Final y_train shape:", y_train.shape)
print("Final x_valid shape:", x_valid.shape)
print("Final y_valid shape:", y_valid.shape)
print("Final x_test shape:", x_test.shape)
print("Final y_test shape:", y_test.shape)
print("Final R_train shape:", R_train.shape)
print("Final R_valid shape:", R_valid.shape)
print("Final R_test shape:", R_test.shape)
print("Final RR_train shape:", RR_train.shape)
print("Final RR_valid shape:", RR_valid.shape)
print("Final RR_test shape:", RR_test.shape)

classes=np.unique(y_train)
print(classes)
print(session_path)
print ('check shapes: ', x_train.shape, y_train.shape, x_test.shape, y_test.shape)

train_lab_categorical = tf.keras.utils.to_categorical(y_train, num_classes=len(np.unique(y_train)))
valid_lab_categorical = tf.keras.utils.to_categorical(y_valid, num_classes=len(np.unique(y_train)))
test_lab_categorical = tf.keras.utils.to_categorical(y_test, num_classes=len(np.unique(y_train)))

print("Check composition of datasets")
print("Train",np.sum(train_lab_categorical.swapaxes(1,0)[0]), np.sum(train_lab_categorical.swapaxes(1,0)[1]), np.sum(train_lab_categorical.swapaxes(1,0)[2]), np.sum(train_lab_categorical.swapaxes(1,0)[3]), np.sum(train_lab_categorical.swapaxes(1,0)[4]))
print("Valid",np.sum(valid_lab_categorical.swapaxes(1,0)[0]), np.sum(valid_lab_categorical.swapaxes(1,0)[1]), np.sum(valid_lab_categorical.swapaxes(1,0)[2]), np.sum(valid_lab_categorical.swapaxes(1,0)[3]), np.sum(valid_lab_categorical.swapaxes(1,0)[4]))
print("Test",np.sum(test_lab_categorical.swapaxes(1,0)[0]), np.sum(test_lab_categorical.swapaxes(1,0)[1]), np.sum(test_lab_categorical.swapaxes(1,0)[2]), np.sum(test_lab_categorical.swapaxes(1,0)[3]), np.sum(test_lab_categorical.swapaxes(1,0)[4]))

training_data = tf.data.Dataset.from_tensor_slices(x_train)
training_RR = tf.data.Dataset.from_tensor_slices(RR_train)
train_labels = tf.data.Dataset.from_tensor_slices(train_lab_categorical)
validation_data = tf.data.Dataset.from_tensor_slices(x_valid)
valid_RR = tf.data.Dataset.from_tensor_slices(RR_valid)
valid_labels = tf.data.Dataset.from_tensor_slices(valid_lab_categorical)
test_data = tf.data.Dataset.from_tensor_slices(x_test)
test_RR = tf.data.Dataset.from_tensor_slices(RR_test)
test_labels = tf.data.Dataset.from_tensor_slices(test_lab_categorical)

autotune = tf.data.AUTOTUNE 

X_t = tf.data.Dataset.zip((training_data, training_RR)).map(lambda x1, x2: {'x1': x1, 'x2': x2}) #map two different inputs, x2 is the RR features
X_v = tf.data.Dataset.zip((validation_data, valid_RR)).map(lambda x1, x2: {'x1': x1, 'x2': x2})
X_T = tf.data.Dataset.zip((test_data, test_RR)).map(lambda x1, x2: {'x1': x1, 'x2': x2})


XY_t = tf.data.Dataset.zip((X_t, train_labels))
XY_v = tf.data.Dataset.zip((X_v, valid_labels))
XY_T = tf.data.Dataset.zip((X_T, test_labels))

train_ds = (XY_t.batch(batch_size).map(lambda x, y: (x, y), num_parallel_calls=autotune).prefetch(autotune))
valid_ds = (XY_v.batch(batch_size).map(lambda x, y: (x, y), num_parallel_calls=autotune).prefetch(autotune))
test_ds = (XY_T.batch(batch_size).map(lambda x, y: (x, y), num_parallel_calls=autotune).prefetch(autotune))


In [ ]:

#############################################################################################################
#Model definition

def generate_patch_conv_orgPaper_f(embedding, inputs):
  print(inputs.shape) 
  patches = layers.Conv2D(filters=embedding, kernel_size=[kernel,1], strides=[kernel,1], padding='valid',input_shape=inputs.shape[1:])(inputs) 
  for i in range(emb_depth-1):
    patches = layers.BatchNormalization()(patches) 
    patches = layers.Conv2D(filters=embedding, kernel_size=[kernel,1], strides=[kernel,1], padding='valid',input_shape=patches.shape)(patches)  
  row_axis, col_axis = (1, 2) # channels last images
  print(patches.shape)
  seq_len = (patches.shape[row_axis]) * (patches.shape[col_axis])
  print(seq_len)
  x = tf.reshape(patches, [-1, seq_len, embedding])
  print(x.shape)
  return x


class AddPositionEmbs(layers.Layer):
  """Adds (optionally learned) positional embeddings to the inputs."""

  def __init__(self, posemb_init=None, **kwargs):
    super().__init__(**kwargs)
    self.posemb_init = posemb_init
    #self.posemb_init=[tf.keras.initializers.RandomNormal(stddev=0.02), name='posembed_input'] # used in original code

  def get_config(self):
        config = super().get_config().copy()
        config.update({
            'posembed_input': self.posemb_init
        })
        return config

  def build(self, inputs_shape):
    pos_emb_shape = (1, inputs_shape[1], inputs_shape[2])
    self.pos_embedding = self.add_weight('pos_embedding', pos_emb_shape, initializer=self.posemb_init)

  def call(self, inputs, inputs_positions=None):
    # inputs.shape is (batch_size, seq_len, emb_dim).
    pos_embedding = tf.cast(self.pos_embedding, inputs.dtype)

    return inputs + pos_embedding


class CLS_Token(layers.Layer):
  """Adds (optionally learned) positional embeddings to the inputs."""

  def __init__(self, token_init=None, **kwargs):
    super().__init__(**kwargs)
    self.token_init = token_init


  def get_config(self):
        config = super().get_config().copy()
        config.update({
            'token_input': self.token_init
        })
        return config

  def build(self, inputs_shape):
    token_shape = (1, 1, inputs_shape[2])
    print(token_shape)
    self.token = self.add_weight('token', token_shape, initializer=self.token_init,trainable=True)
    super(CLS_Token, self).build(inputs_shape)

  def call(self, inputs, inputs_positions=None):
    cls_token = tf.cast(self.token, inputs.dtype)
    cls_token_new = repeat_const(inputs, cls_token) 
    return tf.concat((cls_token_new, inputs), axis=1)


def mlp_block_f(mlp_dim, inputs):
  x = layers.Dense(units=mlp_dim, activation=tf.nn.gelu)(inputs)
  x = layers.Dropout(rate=0.1)(x) # dropout rate is from original paper,
  x = layers.Dense(units=inputs.shape[-1], activation=tf.nn.gelu)(x)
  x = layers.Dropout(rate=0.1)(x)
  return x


def Encoder1Dblock_f_postNorm(num_heads, mlp_dim, inputs):
 # x = layers.LayerNormalization(dtype=inputs.dtype)(inputs) # pre-Norm
 # x = layers.MultiHeadAttention(num_heads=num_heads, key_dim=hidden_size, dropout=0.1)(x, x) # pre-Norm #self attention multi-head, dropout_rate is from original implementation
 # x = layers.MultiHeadAttention(num_heads=num_heads, key_dim=inputs.shape[-1], dropout=0.1)(x, x) # self attention multi-head, dropout_rate is from original implementation
  x = inputs # post-Norm
  x = layers.MultiHeadAttention(num_heads=num_heads, key_dim=hidden_size, dropout=0.1)(x, x) # post-Norm #self attention multi-head, dropout_rate is from original implementation
  x = layers.LayerNormalization(dtype=x.dtype)(x) # post-Norm
  x = layers.Add()([x, inputs]) # 1st residual part 
 # y = layers.LayerNormalization(dtype=x.dtype)(x) # pre-Norm
 # y = mlp_block_f(mlp_dim, y) #pre-Norm
  y = mlp_block_f(mlp_dim, x) #post-Norm
  y = layers.LayerNormalization(dtype=y.dtype)(y) # post-Norm
  y_1 = layers.Add()([y, x]) #2nd residual part 
  return y_1


def Encoder1Dblock_f(num_heads, mlp_dim, inputs):
  x = layers.LayerNormalization(dtype=inputs.dtype)(inputs)
 # x = layers.MultiHeadAttention(num_heads=num_heads, key_dim=inputs.shape[-1], dropout=0.1)(x, x) # self attention multi-head, dropout_rate is from original implementation
  x = layers.MultiHeadAttention(num_heads=num_heads, key_dim=hidden_size, dropout=0.1)(x, x) # self attention multi-head, dropout_rate is from original implementation
  x = layers.Add()([x, inputs]) # 1st residual part 
  y = layers.LayerNormalization(dtype=x.dtype)(x)
  y = mlp_block_f(mlp_dim, y)
  y_1 = layers.Add()([y, x]) #2nd residual part 
  return y_1



def Encoder_f(num_layers, mlp_dim, num_heads, inputs):
  x = AddPositionEmbs(posemb_init=tf.keras.initializers.RandomNormal(stddev=0.02), name='posembed_input')(inputs)
  x = layers.Dropout(rate=0.2)(x)
  for _ in range(num_layers):
   # x = Encoder1Dblock_f(num_heads, mlp_dim, x)
    x = Encoder1Dblock_f_postNorm(num_heads, mlp_dim, x)

  encoded = layers.LayerNormalization(name='encoder_norm')(x)
  return encoded

######################################

def repeat_const(tensor, myconst):
    shapes = tf.shape(tensor)
    return tf.repeat(myconst, shapes[0], axis=0)

def read_cls_token(tensor, myconst):
    #temp = tensor[1:]
    print(tensor.shape)
    myconst=tensor[:,0]
    return myconst

def build_ViT(use_cls_token, cat_RR):
  inputs = layers.Input(shape=x_train.shape[1:],name='x1')
  if cat_RR:
    RR_feat = layers.Input(shape=RR_train.shape[1:],name='x2')
    print("shape of RR feat",RR_train.shape[1:])
    print(type(RR_feat))
  patches = generate_patch_conv_orgPaper_f(embedding, inputs)
  print(type(patches))
  if use_cls_token: 
    patches_new = CLS_Token(token_init=tf.keras.initializers.zeros(), name='token_input')(patches)   
  else:
    patches_new = patches
  encoder_out = Encoder_f(transformer_layers, mlp_dim, num_heads, patches_new)  
  if use_cls_token:
    print(encoder_out.shape)
    im_representation = encoder_out[:,0]
  else:
    im_representation = tf.reduce_mean(encoder_out, axis=1)
  if cat_RR:
    im_representation = tf.concat([im_representation,RR_feat],axis=-1)  

  logits = layers.Dense(units=len(class_types), name='head', kernel_initializer=tf.keras.initializers.zeros())(im_representation) 
  print(logits.shape)

  final_model = tf.keras.Model(inputs = [inputs,RR_feat], outputs = logits)
  return final_model

ViT_model = build_ViT(use_cls_token,cat_RR)
ViT_model.summary()

In [9]:

import keras.backend as K
class CustomCallback(keras.callbacks.Callback):

   def on_train_begin(self, logs=None): # this runs on the beginning of training
        f = open(session_path + "lr_log.txt","r")
        if os.path.getsize(session_path + "lr_log.txt") > 0:
             print("Found file ")
             for line in f:
               pass
             last_line = line
             print(last_line)
             words = line.split()
             print(words)
             last_lr = float(words[-1])
             print("last_lr",last_lr)
             tf.keras.backend.set_value(self.model.optimizer.lr, float(last_lr)) # set the learning rate in the optimizer
        f.close()
        #tf.keras.backend.set_value(self.model.optimizer.lr, 5e-6) # set the learning rate in the optimizer
        print("after log file read")
        lr=float(tf.keras.backend.get_value(self.model.optimizer.lr)) # get the current learning rate to insure it is set
        print('Optimizer learning rate set to ', lr)

   def on_epoch_begin(self, epoch, logs=None):
        keys = list(logs.keys())
        print("Start epoch {} of training; got log keys: {}".format(epoch, keys))
        lr = K.eval(self.model.optimizer.lr.numpy())
        print("\nLearning rate at epoch {} is {}".format(epoch, lr))        
        f = open(session_path + "lr_log.txt","a")
        f.write("\nLearning rate at epoch {} is {}".format(epoch, lr))
        f.close()


   def on_epoch_end(self, epoch, logs=None):
        keys = list(logs.keys())
        print("End epoch {} of training; train_loss: {:1.4f}, train_accuracy: {:2.4f}, validation_loss: {:1.4f}, validation_accuracy: {:2.4f}\n".format(epoch, logs["loss"], logs["accuracy"], logs["val_loss"], logs["val_accuracy"]))
        f = open(session_path + "train_log.txt","a")
        f.write("End epoch {} of training; train_loss: {:1.4f}, train_accuracy: {:2.4f}, validation_loss: {:1.4f}, validation_accuracy: {:2.4f}\n".format(epoch, logs["loss"], logs["accuracy"], logs["val_loss"], logs["val_accuracy"]))
        f.close()

In [10]:

def print_stats(predictions, labels):
  print("Accuracy = {}".format(accuracy_score(labels, predictions)))
  print("Precision = {}".format(precision_score(labels, predictions,average="micro")))
  print("Recall = {}".format(recall_score(labels, predictions,average="micro")))

def conf_matrix_ecg(predictions,y_test,class_types, session_path): 
    ''' Plots conf. matrix and classification report '''
    print(class_types, len(class_types), y_test.shape)
    cm=confusion_matrix(y_test, np.argmax(np.round(predictions), axis=1),labels= np.arange(len(class_types)))
    print(cm)
    print_stats(np.argmax(np.round(predictions), axis=1), y_test)
    plt.figure(figsize=(12,12))
    sns_hmp = sns.heatmap(cm, annot=True, xticklabels = [class_types[i] for i in range(len(class_types))], 
                yticklabels = [class_types[i] for i in range(len(class_types))], fmt="d")
    fig = sns_hmp.get_figure()
    fig.savefig(session_path+'confusion_matrix.png', dpi=250)
    cr=classification_report(y_test,
                                np.argmax(np.round(predictions), axis=1),labels= np.arange(len(class_types)), 
                                target_names=[class_types[i] for i in range(len(class_types))])
    print(cr)
    print("\n\nMatteo metrics\n")
    print("\n\nTN FP FN TP\n")
    n_rows = len(cm)
    TN = cm[0][0]
    FP = 0
    FN = 0
    TP = 0
    for i in range(n_rows):
      if i > 0:
        TP = TP + cm[i][i]
        FN = FN + cm[i][0]
        FP = FP + cm[0][i] 
        if i != 1:
          FP = FP + cm[1][i]
        if i != 2:
          FP = FP + cm[2][i]
        if i != 3:
          FP = FP + cm[3][i]
    print(TN, FP, FN, TP)
    accuracy, sensitivity, specificity, precision, f1_score = calculate_metrics_from_cm(TN, FP, FN, TP)
    print("Accuracy Sensitivity Specificity Precision F1\n")
    print(str(accuracy) + " " + str(sensitivity) + " " + str(specificity) + " " + str(precision) + " " + str(f1_score)+"\n")
    return accuracy

def calculate_metrics_from_cm(TN, FP, FN, TP):
    if(TP + FN != 0):
        sensitivity = (TP / (TP + FN)) * 100
    else:
        sensitivity = 0
    if(TN + FP != 0):
        specificity = (TN / (TN + FP)) * 100
    else:
        specificity = 0
    accuracy = (TP + TN) / (TN + FN + TP + FP) * 100
    if(TP + FP != 0):    
        precision = (TP / (TP + FP)) * 100
    else:
        precision = 0
    print(str(precision>0))
    print(str(sensitivity>0.0))
    if precision > 0 or sensitivity > 0.0:
        f1_score = (2 * ((precision*sensitivity)/(precision+sensitivity)))
    else:    
        f1_score = 0 
    return accuracy, sensitivity, specificity, precision, f1_score



In [ ]:
filepath = session_path+ "ecg_transformer_acc_"
filepath_loss = session_path+ "ecg_transformer_loss_"
filepath_curr = session_path+ "ecg_transformer_curr.h5"

if not os.path.exists(session_path + "train_log.txt"):
    f = open(session_path + "train_log.txt","w")
    f.close()
    f = open(session_path + "lr_log.txt","w")
    f.close()
    epoch_done = 0
else:
    model = session_path + "ecg_transformer_curr.h5"
    ViT_model.load_weights(model)
    print('model exist')

In [ ]:
ViT_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=2e-3), 
                  loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True), 
                  metrics=[tf.keras.metrics.CategoricalAccuracy(name="accuracy"), tf.keras.metrics.TopKCategoricalAccuracy(k=5, name='top5 acc')]) 

#queste no :)
#old_weights = ViT_model.get_weights()
#print(len(old_weights))
#print(range(len(old_weights)))
#for i in range(len(old_weights))[:-2]:  #load weights of the model pretrained on AFIB dataset
#  old_weights[i] = np.load("output/train_raw/model6_7_1_2_atrialpretr/" + str(i) + "AFIB_weights.npy")
#  print(str(i) + "AFIB_weights.npy")
#ViT_model.set_weights(old_weights)
#ViT_model.save(session_path + "start_model.h5")

checkpoint = ModelCheckpoint(filepath + "{epoch:02d}-{val_accuracy:.4f}.h5", monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
checkpoint_loss = ModelCheckpoint(filepath_loss + "{epoch:02d}-{val_loss:.4f}.h5", monitor='val_loss', verbose=1, save_best_only=True, mode='min')
checkpoint_curr = ModelCheckpoint(filepath_curr, monitor='val_loss', verbose=1, save_best_only=False, mode='min')

writelog = CustomCallback()
print("starting checkpoint",checkpoint.best)

f = open(session_path + "train_log.txt","r")
if os.path.getsize(session_path + "train_log.txt") > 0:
    print("Found file ")
    i = 0
    for n, line in enumerate(f.readlines()):
            split1, split2, split3, split4 = line.split(",")
            epoch_done = n + 1 # int(re.search(r'\d+', split1[10:15]).group()) + 1
            train_loss = float(split1[-5:])
            validation_loss = float(split3[-5:])
            train_accuracy = float(split2[-6:])
            validation_accuracy = float(split4[-6:])
            if i == 0:
               best = validation_accuracy
               best_loss = validation_loss
               i = i + 1
            elif validation_accuracy > best:
               best = validation_accuracy
            if validation_loss < best_loss:
               best_loss = validation_loss

    checkpoint.best = best
    checkpoint_loss.best = best_loss
    print("new checkpoint is ",checkpoint.best, " and ", checkpoint_loss.best)

#results = ViT_model.evaluate(valid_ds, batch_size=16)
#print(results, ViT_model.metrics_names)
#checkpoint.best = results[1]
#checkpoint_loss.best = results[0]

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.8,
                              patience=5, min_lr=1e-7, verbose=1)

print(train_ds)
print(valid_ds)
print(y_train.shape)
class_weights = class_weight.compute_class_weight('balanced',
                                                 classes=np.unique(y_train),
                                                 y=y_train)

class_weights = dict(zip(np.unique(y_train), class_weights))
ViT_Train = ViT_model.fit(train_ds, 
                       epochs = 300 - epoch_done, #aggiorna il numero di epoche residue se si interrompe il runtime
                       validation_data=valid_ds, callbacks=[reduce_lr, checkpoint, checkpoint_loss, checkpoint_curr, writelog])#,class_weight=class_weights)



In [13]:
def get_file_with_highest_accuracy(folder_path):
    pattern = re.compile(r'ecg_transformer_acc_(\d+)-(\d+\.\d+)')
    max_accuracy = 0
    max_accuracy_file = None

    for file in os.listdir(folder_path):
        match = pattern.match(file)
        if match:
            accuracy = float(match.group(2))
            if accuracy > max_accuracy:
                max_accuracy = accuracy
                max_accuracy_file = file

    return max_accuracy_file

file_with_highest_accuracy = get_file_with_highest_accuracy(session_path)

**Test**

In [ ]:
filepath = session_path + file_with_highest_accuracy #f"ecg_transformer_acc_04-0.9621.h5" #il test va aggiornato di volta in volta con il nome del modello da testare
ViT_model.load_weights(filepath)


#load best model
ViT_model.load_weights(filepath)


pred_class_eeg = ViT_model.predict(test_ds, batch_size=1)

res_log = open("res_log_keras_out.txt","w")
print(pred_class_eeg.shape)

for i in range(pred_class_eeg.shape[0]):
  res_log.write("\nout: " + str(pred_class_eeg[i]) + " class: " + str(np.argmax(np.round(pred_class_eeg[i]))))    
#  res_log.write("\nclass: " + str(np.argmax(np.round(pred_class_eeg[i])))) 
 # print(i)   
  if np.argmax(np.round(pred_class_eeg[i])) != y_test[i]:
   print("predicted: ",np.argmax(np.round(pred_class_eeg[i])), " truth: ",y_test[i], " index: ",i)

conf_matrix_ecg(pred_class_eeg, y_test,class_types,session_path)